# Imports

In [1]:
import torch
from torch.nn.utils import prune

from tqdm import tqdm

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from utils.mask_utils import calculate_mask
from utils.hessian_utils import calc_inverse_hessian
from utils.prehook_utils import put_input_hooks
import gc

# Constants

In [6]:
CALIBRATION_SIZE=128
TOKEN_LENGTH=1024
CALIBRATION_BATCH_SIZE=1

EPSILON = 1e-8
B = 128 
Bs = 128

#hyperparam test, remove later
EPOCH_COUNT = 10

#set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' 
MODEL_NAME = "opt-125m"

#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'facebook/{MODEL_NAME}')
#Load dataset
dataset = load_dataset('c4', 'en', streaming=True)

# Calibration

## Function

In [4]:
# run model on batches of calibration data, then concatenate inputs
def split_model_calibration(model):
    batch_sentences = []
    for i, data in tqdm(enumerate(iter(dataset['train'])), total=CALIBRATION_SIZE):
        if i < CALIBRATION_SIZE + 1:
            if len(batch_sentences) >= CALIBRATION_BATCH_SIZE:
                with torch.no_grad():
                    encoded_input = tokenizer(batch_sentences, return_tensors="pt",
                                              padding="max_length", max_length=TOKEN_LENGTH,
                                              truncation=True).to(device=DEVICE)
                    model(**encoded_input, labels=encoded_input.input_ids)
                    del encoded_input
                    torch.cuda.empty_cache()
                    batch_sentences = []
            batch_sentences.append(data['text'])
        else:
            break

# SparseGPT

## Prune models (SparseGPT)

In [ ]:
from utils.prune_utils import sparsegpt_prune

SPARSITIES = [0.2,0.3,0.5,0.7,0.9,1]#0.1, 0.2,0.3,0.5,0.7,0.9,1
    
for i, SPARSITY in enumerate(tqdm(SPARSITIES, total=len(SPARSITIES))):
    model = OPTForCausalLM.from_pretrained(f'facebook/{MODEL_NAME}', 
                                       output_attentions=True, 
                                       output_hidden_states=True).to(device=DEVICE)
    model = torch.nn.DataParallel(model, device_ids=[0,1,2,3])
    !nvidia-smi
    # Calculate feature hessians only for the first iteration
    if i == 0:
        feature_hessians = {}
        #put_input_hooks(model=model, features=feature_hessians, storage_dir=storage_dir, offload_freq=10000, feature_storage_device='cpu')
        all_hooks = put_input_hooks(model=model, features=feature_hessians, feature_storage_device='cpu')
        split_model_calibration(model)
        for hook in all_hooks:
            hook.remove()
    
    # Prune using the sparseGPT method, saves as pruned_models/{model_name}-{SPARSENESS}.pt WITHOUT mask
    sparsegpt_prune(model, MODEL_NAME, feature_hessians, EPSILON, SPARSITY, B, Bs)
    !nvidia-smi
    del model
del feature_hessians

  0%|          | 0/6 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Sat Feb 25 13:12:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   28C    P0    60W / 250W |  26355MiB / 40960MiB |     43%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   


  1%|          | 1/128 [00:00<00:30,  4.17it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

 51%|█████     | 65/128 [24:07<23:55, 22.79s/it]

## Finetune models (SparseGPT)

In [16]:
!pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 199 kB 28.4 MB/s eta 0:00:01
You should consider upgrading via the '/gs/gsfs0/hpc01/rhel8/apps/conda3/bin/python3.9 -m pip install --upgrade pip' command.


In [5]:
from utils.finetune_utils import finetune_model
SPARSITIES = [0.2,0.3,0.5,0.7,0.9,1]#0.1, 0.2,0.3,0.5,0.7,0.9,1
    
for i, SPARSITY in enumerate(tqdm(SPARSITIES, total=len(SPARSITIES))):
    # Finetune, saves as pruned_models/{model_name}-{SPARSENESS}-finetuned.pt WITHOUT mask
    finetune_model(MODEL_NAME, tokenizer, SPARSITY, EPOCH_COUNT=EPOCH_COUNT)

  0%|          | 0/10 [00:00<?, ?it/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  0%|          | 0/6 [00:26<?, ?it/s]


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /tmp/ipykernel_28356/505065541.py:6 in <cell line: 4>                                     │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_28356/505065541.py'                  │
│                                                                                           │
│ /gs/gsfs0/home/asyed/pw/jobs/ICLR/utils/finetune_utils.py:53 in finetune_model            │
│                                                                                           │
│   50 │   │   │   outputs = loaded_model(**batch)                                          │
│   51 │   │   │   loss = outputs[0]                                                        │
│   52 │   │   │   loss.backward()                                                          │
│ ❱ 53 │   │   │   t_optim.step()                                                           │
│   54 │   │   │   t_optim.zero_grad()                                                      │
│   55 │   unmask_model(loaded_model)                                                       │
│   56 │   torch.save(loaded_model.state_dict(), f'pruned_models/{model_name}-{SPARSITY}-fi │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/optim/optimizer.py:140 in  │
│ wrapper                                                                                   │
│                                                                                           │
│   137 │   │   │   │   obj, *_ = args                                                      │
│   138 │   │   │   │   profile_name = "Optimizer.step#{}.step".format(obj.__class__.__name │
│   139 │   │   │   │   with torch.autograd.profiler.record_function(profile_name):         │
│ ❱ 140 │   │   │   │   │   out = func(*args, **kwargs)                                     │
│   141 │   │   │   │   │   obj._optimizer_step_code()                                      │
│   142 │   │   │   │   │   return out                                                      │
│   143                                                                                     │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/autograd/grad_mode.py:27   │
│ in decorate_context                                                                       │
│                                                                                           │
│    24 │   │   @functools.wraps(func)                                                      │
│    25 │   │   def decorate_context(*args, **kwargs):                                      │
│    26 │   │   │   with self.clone():                                                      │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                        │
│    28 │   │   return cast(F, decorate_context)                                            │
│    29 │                                                                                   │
│    30 │   def _wrap_generator(self, func):                                                │
│                                                                                           │
│ /gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/optim/adamw.py:149 in step │
│                                                                                           │
│   146 │   │   │   │   │   # Exponential moving average of gradient values                 │
│   147 │   │   │   │   │   state['exp_avg'] = torch.zeros_like(p, memory_format=torch.pres │
│   148 │   │   │   │   │   # Exponential moving average of squared gradient values         │
│ ❱ 149 │   │   │   │   │   state['exp_avg_sq'] = to

In [1]:
!nvidia-smi

Sun Feb 26 03:34:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   28C    P0    59W / 250W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-PCI...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   

## Test models (SparseGPT)

In [3]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aaquib111. Use `wandb login --relogin` to force relogin


True

In [4]:
import numpy as np
from testing_module import test_model

TOKEN_LENGTH=1024
STRIDE = 512
wandb.init(project="ICLR", 
           name = f'{MODEL_NAME} Wikitext Test', 
           config={'token_length': TOKEN_LENGTH,
                 'model_name': MODEL_NAME,
                 'stride': STRIDE,
                 'fine_tuned': 'not finetuned'})
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'facebook/{MODEL_NAME}', 
                                          padding_side='left', 
                                          use_fast=False)
# Load dataset
test_set = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer("\n\n".join(test_set['text']), return_tensors='pt')

seq_len = encodings.input_ids.size(1)
SPARSITIES = [0.2, 0.3, 0.5, 0.7, 0.9, 1]#, 0.4, 0.6, 0.8, 1

for SPARSITY in SPARSITIES:
    test_model(MODEL_NAME, encodings, TOKEN_LENGTH, seq_len, STRIDE, wandb, SPARSITY, is_finetuned=False)
    
### NOW DO FINETUNED
wandb.init(project="ICLR", 
           name = f'{MODEL_NAME} Wikitext Test', 
           config={'token_length': token_length,
                 'model_name': MODEL_NAME,
                 'stride': stride,
                 'fine_tuned': 'finetuned'})
for SPARSITY in SPARSITIES:
    test_model(MODEL_NAME, encodings, TOKEN_LENGTH, seq_len, STRIDE, wandb, SPARSITY, is_finetuned=True)

Found cached dataset wikitext (/gs/gsfs0/users/asyed/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
  0%|          | 0/562 [00:00<?, ?it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
  3%|▎         | 18/562 [00:04<02:11,  4.13it/s]

KeyboardInterrupt



In [7]:
import numpy as np
from utils.test_utils import test_model

TOKEN_LENGTH=1024
STRIDE = 512
wandb.init(project="ICLR", 
           name = f'{MODEL_NAME} Wikitext Test', 
           config={'token_length': TOKEN_LENGTH,
                 'model_name': MODEL_NAME,
                 'stride': STRIDE,
                 'fine_tuned': 'iterative'})
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(f'facebook/{MODEL_NAME}', 
                                          padding_side='left', 
                                          use_fast=False)
# Load dataset
test_set = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer("\n\n".join(test_set['text']), return_tensors='pt')

seq_len = encodings.input_ids.size(1)
SPARSITIES = [0.2, 0.3, 0.5, 0.7, 0.9, 1]#, 0.4, 0.6, 0.8, 1

for SPARSITY in SPARSITIES:
    test_model(MODEL_NAME, encodings, TOKEN_LENGTH, seq_len, STRIDE, wandb, SPARSITY, model_type='iterative')

Found cached dataset wikitext (/gs/gsfs0/users/asyed/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
  0%|          | 0/562 [00:00<?, ?it/s]/gs/gsfs0/users/asyed/.local/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|█████████▉| 560/562 [00:19<00:00, 28.82it/s]


# Cerebras

In [ ]:
for model_name in ['opt-125m', 'opt-350m', 'opt-1.3b', 'opt-2.7b']:
    for sparsity in [0.2,0.3,0.5,0.7,0.9,1]:
        